Carteira baseada no número de Graham sobre o valor intrínseco de uma ação, foca em pegar empresas que sejam lucrativas e baratas: empresas operando a P/L menor que 15 e com P/VPA menor que 1,5. Multiplicando 15 x 1,5 temos o número 22,5 que nos indica a fórmula de graham. Valor Intrínseco de uma ação = $\sqrt{22,5 \times \text{LPA} \times \text{VPA}}$.

Critérios:


- Ter Lucro por ação maior que zero ,isto é, a empresa não pode estar com prejuízo atualmente
- Ter Valor Patrimonial por ação positivo, ou seja, a empresa não pode ter mais passivos (obrigações a pagar) que ativos (bens ou direitos a receber)
- Ter volume médio de negociação diário de no mínimo R$250.000,00
- Ter lucro líquido medio positivo em todos os últimos 5 exercícios.


In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from IPython.display import display, HTML

def b_print(df , n=30 , clean=True): #beauty print :)
    
    # from IPython.display import display, HTML

    # if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
    #     df['prefixo'] = df['Papel'].astype(str).str[:4]
    #     df=df.drop_duplicates(subset='prefixo', keep='first')
    #     # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None


In [2]:
from DT_atualiza_settings import *
from DT_StatusInvest import SI
SI(mercado = 'Acoes' )

 ====== SI Acoes ===== 
 ====== Busca
 ====== Download 


In [4]:
import os
# Caminho do arquivo local
file_path = os.path.expanduser('~/GHub/Finance-playground/data/SI_Acoes.csv')
# URL para o arquivo online
file_url = 'https://raw.githubusercontent.com/BDonadelli/Finance-playground/refs/heads/main/data/SI_Acoes.csv'

# Verificar se o arquivo existe localmente
if os.path.exists(file_path):
    # Ler o arquivo local
    funds = pd.read_csv(file_path,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido localmente.")
else:
    # Ler o arquivo a partir da URL
    funds = pd.read_csv(file_url,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido da URL.")


Arquivo lido localmente.


In [5]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [6]:
fundsSI =  funds[ (funds[' LIQUIDEZ MEDIA DIARIA'] > 300000) &
                  (funds[' LPA'] > 0) & 
                  (funds[' VPA'] > 0) & 
                  (funds['CAGR LUCROS 5 ANOS'] > 0) ]
fundsSI

fundsSI['valor intrinseco'] = np.round(np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA']),2)
fundsSI['VALOR DE MERCADO (em B)'] = fundsSI[' VALOR DE MERCADO']  / 1e9
fundsSI['LIQUIDEZ MEDIA DIARIA (em M)'] = fundsSI[' LIQUIDEZ MEDIA DIARIA'] / 1e6
fundsSI['Delta (%)'] = np.round((fundsSI['valor intrinseco'] / fundsSI['PRECO'] -1)*100,2)
fundsSI["Rank"]   = fundsSI['Delta (%)'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

colunas_exibidas = ['TICKER','PRECO' , 'CAGR LUCROS 5 ANOS' ,'LIQUIDEZ MEDIA DIARIA (em M)',
                 ' VPA',' LPA' ,  'VALOR DE MERCADO (em B)',
                'valor intrinseco' , 'Delta (%)' ,'P/L', 'DIVIDA LIQUIDA / EBIT' ]

b_print(fundsSI[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
LIGT3,5.91,12.76,4.640564,15.14,6.50,2.201802,47.06,696.28,0.91,3.87
SYNE3,6.73,76.27,2.872464,7.15,3.66,1.027297,24.27,260.62,1.84,0.48
BAZA3,72.51,34.96,0.444669,119.36,21.99,4.064788,243.02,235.15,3.30,NaN
JHSF3,5.17,27.06,8.541291,8.44,1.57,3.522439,17.27,234.04,3.30,2.09
COGN3,2.75,33.12,55.867468,6.63,0.52,5.160667,8.81,220.36,5.25,3.83
LOGG3,20.42,32.94,3.968972,41.80,4.26,1.794084,63.30,209.99,4.79,3.59
SOMA3,5.95,80.87,99.716529,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
BBAS3,18.42,8.51,541.086402,31.72,4.30,105.561963,55.40,200.76,4.28,NaN
MTRE3,3.67,13.19,1.303651,9.43,0.56,0.388194,10.90,197.00,6.54,4.88
CAML3,4.53,2.67,6.065228,9.88,0.78,1.585500,13.17,190.73,5.80,5.02


outros criterios adicionais

P/L > 0 e DIVIDA LIQUIDA / EBIT < 3

In [7]:
'''
P/L > 0 e DIVIDA LIQUIDA / EBIT < 3
'''
fundsSI.fillna(0,inplace=True) ## bancos tem NaN
b_print(fundsSI[(fundsSI['P/L'] > 0) & (fundsSI['DIVIDA LIQUIDA / EBIT'] < 3) ][colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
SYNE3,6.73,76.27,2.872464,7.15,3.66,1.027297,24.27,260.62,1.84,0.48
BAZA3,72.51,34.96,0.444669,119.36,21.99,4.064788,243.02,235.15,3.30,0.00
JHSF3,5.17,27.06,8.541291,8.44,1.57,3.522439,17.27,234.04,3.30,2.09
SOMA3,5.95,80.87,99.716529,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
BBAS3,18.42,8.51,541.086402,31.72,4.30,105.561963,55.40,200.76,4.28,0.00
ISAE4,21.57,13.00,26.093548,31.27,4.93,16.515497,58.90,173.06,4.38,2.57
USIM5,4.37,12.02,48.854461,19.37,0.30,5.602903,11.43,161.56,14.56,0.96
SAPR4,6.59,17.06,5.621539,7.97,1.57,10.315489,16.78,154.63,4.20,1.76
USIM3,4.55,12.02,1.423230,19.37,0.30,5.602903,11.43,151.21,15.16,0.96
GOAU4,8.99,21.50,42.943014,19.50,1.13,8.969926,22.27,147.72,7.94,1.52


remove empresas repetidas, mantem primeira ocorrencia

In [8]:
fundsSI['prefixo'] = fundsSI['TICKER'].str[:4]
fundsSI.drop_duplicates(subset='prefixo', keep='first').head(20)
df_limpo = fundsSI.drop_duplicates(subset='prefixo', keep='first')
df_limpo = df_limpo.drop('prefixo', axis=1) 
b_print(df_limpo[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA (em M),VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
LIGT3,5.91,12.76,4.640564,15.14,6.50,2.201802,47.06,696.28,0.91,3.87
SYNE3,6.73,76.27,2.872464,7.15,3.66,1.027297,24.27,260.62,1.84,0.48
BAZA3,72.51,34.96,0.444669,119.36,21.99,4.064788,243.02,235.15,3.30,0.00
JHSF3,5.17,27.06,8.541291,8.44,1.57,3.522439,17.27,234.04,3.30,2.09
COGN3,2.75,33.12,55.867468,6.63,0.52,5.160667,8.81,220.36,5.25,3.83
LOGG3,20.42,32.94,3.968972,41.80,4.26,1.794084,63.30,209.99,4.79,3.59
SOMA3,5.95,80.87,99.716529,7.40,2.01,4.669247,18.29,207.39,2.96,-0.45
BBAS3,18.42,8.51,541.086402,31.72,4.30,105.561963,55.40,200.76,4.28,0.00
MTRE3,3.67,13.19,1.303651,9.43,0.56,0.388194,10.90,197.00,6.54,4.88
CAML3,4.53,2.67,6.065228,9.88,0.78,1.585500,13.17,190.73,5.80,5.02


### outra estória de porque 22,5

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce. Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos. O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores. Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.

# Stocks

In [14]:
import os
# Caminho do arquivo local /home/yair/GHub/Codigos-em-financas/data/SI_Stocks.csv
file_path = os.path.expanduser('/home/yair/GHub/Finance-playground/data/SI_Stocks.csv')
# URL para o arquivo online
file_url = 'https://raw.githubusercontent.com/BDonadelli/Finance-playground/refs/heads/main/data/SI_Stocks.csv'

# Verificar se o arquivo existe localmente
if os.path.exists(file_path):
    # Ler o arquivo local
    funds = pd.read_csv(file_path,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido localmente.")
else:
    # Ler o arquivo a partir da URL
    funds = pd.read_csv(file_url,sep=';' , decimal=',' ,thousands ='.' )
    print("Arquivo lido da URL.")


Arquivo lido localmente.


In [15]:
funds

,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
0,A,113.32,0.65,27.61,5.25,2.65,53.35,22.36,17.59,21.72,23.08,1.36,0.33,4.86,14.66,-4.05,2.09,19.00,9.59,13.26,0.50,0.50,0.55,-73.69,-73.93,NaN,21.60,4.10,-4.51,3.219021e+10
1,AA,28.41,1.06,8.55,1.26,0.50,20.43,11.56,6.79,5.02,6.72,1.69,0.43,0.58,3.39,-0.79,1.71,14.79,5.90,11.09,0.40,0.59,0.87,2.66,NaN,NaN,22.46,3.32,-0.04,7.355414e+09
2,AACG,1.18,NaN,-3.60,1.63,0.52,51.68,-31.49,-32.34,-3.70,-3.09,0.61,-0.27,1.17,-1.12,-0.60,0.23,-45.26,-14.39,-39.89,0.32,0.68,0.44,21.21,NaN,NaN,0.72,-0.33,-0.09,3.731691e+07
3,AACQ,9.99,NaN,-93.43,180.94,1.25,NaN,NaN,NaN,-27.75,-27.74,0.01,-0.07,NaN,2094.26,-1.25,3.56,-193.66,-1.33,-652.08,0.01,0.12,0.00,NaN,NaN,NaN,0.06,-0.11,NaN,9.047194e+08
4,AACQU,11.05,NaN,-103.35,200.14,1.38,NaN,NaN,NaN,-30.69,-27.74,0.01,-0.07,NaN,2316.47,-1.38,3.56,-193.66,-1.33,-652.08,0.01,0.12,0.00,NaN,NaN,NaN,0.06,-0.11,NaN,9.047194e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5248,ZUO,10.02,NaN,-20.82,8.36,1.80,67.42,-10.54,-16.34,-32.28,-28.30,3.99,-1.03,3.40,3.65,-8.80,2.62,-40.16,-8.63,-8.89,0.21,0.79,0.53,349.59,NaN,NaN,1.20,-0.48,0.40,1.540242e+09
5249,ZVO,0.09,NaN,-0.08,0.47,0.04,35.09,-17.75,-17.80,-0.08,0.59,0.66,-4.11,0.01,-1.38,-0.07,0.95,-619.73,-47.91,-619.73,0.08,0.92,2.69,-12.86,NaN,NaN,0.19,-1.20,0.00,3.078549e+06
5250,ZYME,12.57,NaN,-8.28,2.90,2.21,100.00,-114.77,-121.73,-8.78,-6.30,2.48,-0.82,10.07,3.90,-7.90,4.71,-34.98,-26.71,-34.98,0.76,0.24,0.22,20.90,NaN,NaN,4.34,-1.52,0.85,9.407954e+08
5251,ZYNE,1.30,NaN,-1.87,0.00,0.00,NaN,NaN,NaN,-1.87,955.19,957.05,-1.24,NaN,0.00,-0.06,3.43,-0.13,-0.09,-0.13,0.72,0.28,0.00,NaN,NaN,NaN,538.03,-0.70,4.43,7.012122e+07


In [19]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [24]:
fundsSI =  funds[ (funds[' LPA'] > 1) & 
                  (fundsSI['DIVIDA LIQUIDA / EBIT'] < 4) & 
                  (funds[' VPA'] > 0) 
                #   (funds['CAGR LUCROS 5 ANOS'] > 0) 
                  ]
b_print(fundsSI)

TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,EV/EBIT,DIVIDA LIQUIDA / EBIT,DIV. LIQ. / PATRI.,PSR,P/CAP. GIRO,P. AT CIR. LIQ.,LIQ. CORRENTE,ROE,ROA,ROIC,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
AAON,81.75,0.44,0.05,8.15,5.12,31.04,10.97,10.84,0.05,0.05,0.00,0.31,0.01,18.22,-9.13,2.77,16744.95,10507.21,9815.88,0.63,0.37,969.54,20.67,25.70,NaN,10.03,1679.00,0.00,6.651679e+09
AAPL,203.95,0.50,31.31,45.60,9.20,46.63,31.72,24.30,23.99,24.38,0.39,0.74,7.61,-117.63,-14.33,0.82,145.66,29.37,58.97,0.20,0.80,1.21,8.49,11.15,NaN,4.47,6.51,0.46,3.046162e+12
AAWW,102.48,NaN,0.02,957.44,438.20,73340.41,12124.84,5712.28,0.01,-2.86,-2.88,-252517.36,1.32,-63287.62,-526.68,0.96,4136738.51,1893306.24,4345055.72,0.46,0.54,331.44,16.10,9.75,NaN,0.11,4427.79,0.00,2.934330e+09
ABB,37.12,NaN,17.32,5.68,1.80,32.37,17.56,13.65,13.46,13.66,0.20,0.09,2.36,89.85,-3.35,1.04,32.76,10.36,24.67,0.32,0.68,0.76,3.17,2.26,NaN,6.54,2.14,0.15,6.900207e+10
ABCB,66.12,0.83,12.24,1.19,0.17,0.00,31.82,22.24,8.56,2.67,-5.88,-0.82,2.72,NaN,-0.17,NaN,9.74,1.40,10.09,0.14,0.86,0.06,14.90,17.31,NaN,55.49,5.40,0.39,4.556389e+09
ABMD,381.02,NaN,0.16,11.16,10.13,81.62,21.82,39.85,0.30,0.28,-0.01,-0.53,0.06,18.50,-27.11,7.93,6903.14,6265.30,3565.27,0.91,0.09,157.22,18.30,21.24,NaN,34.15,2357.30,0.00,1.718065e+10


In [25]:
fundsSI['valor intrinseco'] = np.round(np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA']),2)
fundsSI['VALOR DE MERCADO (em B)'] = fundsSI[' VALOR DE MERCADO']  / 1e9
# fundsSI['LIQUIDEZ MEDIA DIARIA (em M)'] = fundsSI[' LIQUIDEZ MEDIA DIARIA'] / 1e6
fundsSI['Delta (%)'] = np.round((fundsSI['valor intrinseco'] / fundsSI['PRECO'] -1)*100,2)
fundsSI["Rank"]   = fundsSI['Delta (%)'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

colunas_exibidas = ['TICKER','PRECO' , 'CAGR LUCROS 5 ANOS' ,
                 ' VPA',' LPA' ,  'VALOR DE MERCADO (em B)',
                'valor intrinseco' , 'Delta (%)' ,'P/L', 'DIVIDA LIQUIDA / EBIT' ]

b_print(fundsSI[colunas_exibidas])

TICKER,PRECO,CAGR LUCROS 5 ANOS,VPA,LPA,VALOR DE MERCADO (em B),valor intrinseco,Delta (%),P/L,DIVIDA LIQUIDA / EBIT
AAON,81.75,25.70,10.03,1679.00,6.651679,615.56,652.98,0.05,0.00
ABMD,381.02,21.24,34.15,2357.30,17.180649,1345.84,253.22,0.16,-0.01
ABCB,66.12,17.31,55.49,5.40,4.556389,82.11,24.18,12.24,-5.88
AAWW,102.48,9.75,0.11,4427.79,2.934330,104.68,2.15,0.02,-2.88
ABB,37.12,2.26,6.54,2.14,69.002075,17.75,-52.18,17.32,0.20
AAPL,203.95,11.15,4.47,6.51,3046.161713,25.59,-87.45,31.31,0.39
